In [60]:
import pandas as pd
import statsmodels.api as sm
stock = input("Enter the stock name: ")
# Replace the path with the correct path to your CSV file
file_path1 = f"D:\\Study Program\\Project\\Price\\{stock}_Price.csv"
file_path2 = f"D:\\Study Program\\Project\\News\\{stock}_news.csv"
# Load the CSV file into a DataFrame
df1 = pd.read_csv(file_path1)
df2 = pd.read_csv(file_path2)
# Convert Date and Time to the same format
df1['Date'] = pd.to_datetime(df1['Date'])
df2['Time'] = pd.to_datetime(df2['Time'], format ="%d/%m/%Y %H:%M")

# Extract the date part from df2['Time']
df2['Time'] = df2['Time'].dt.date

# Perform the left join
merged_df = pd.merge(df1, df2, left_on=df1['Date'].dt.date, right_on='Time', how='left')

# Drop the redundant column 'key_0' if needed
merged_df.drop(columns=['Link', 'Content'], inplace=True)
# Fill NaN values in the 'Evaluation' column with 0
merged_df['Evaluation'].fillna(0, inplace=True)
merged_df = merged_df.sort_values(by='Date')
# Remove commas and convert Price-related columns to float
merged_df['Close'] = merged_df['Price'].str.replace(',', '').astype(float)
merged_df['Open'] = merged_df['Open'].str.replace(',', '').astype(float)
merged_df['High'] = merged_df['High'].str.replace(',', '').astype(float)
merged_df['Low'] = merged_df['Low'].str.replace(',', '').astype(float)
def convert_volume(vol):
    if 'M' in vol:
        return float(vol.replace('M', '')) * 1_000_000
    elif 'K' in vol:
        return float(vol.replace('K', '')) * 1_000
    else:
        return float(vol)

merged_df['Vol.'] = merged_df['Vol.'].apply(convert_volume)
# Insert a new column 'Period' at the first position with values ranging from 1 to the length of the DataFrame
merged_df.insert(0, 'Period', range(1, len(merged_df) + 1))

# Drop the column 'Change %'
df = merged_df.drop(columns=['Change %', "Price"])

df['Month'] = pd.to_datetime(df['Date']).dt.month
# Create dummy variables for 'Month'
df = pd.get_dummies(df, columns=['Month'], drop_first=True)
# Convert boolean columns to numeric (0 and 1) directly
bool_columns = df.select_dtypes(include='bool').columns
df[bool_columns] = df[bool_columns].astype(int)
# Prepare the dependent and independent variables
X = df[['Period'] + [col for col in df.columns if 'Month_' in col]+ ['Evaluation']]
y = df['Close']

# Add a constant to the independent variables
X = sm.add_constant(X)

# Fit the multiple regression model
model = sm.OLS(y, X).fit()

# Print the regression results
print(model.summary())

C:\Users\andoa\AppData\Local\Temp\ipykernel_6052\1720408685.py:23: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





                            OLS Regression Results                            
Dep. Variable:                  Close   R-squared:                       0.731
Model:                            OLS   Adj. R-squared:                  0.730
Method:                 Least Squares   F-statistic:                     989.6
Date:                Fri, 24 May 2024   Prob (F-statistic):               0.00
Time:                        08:33:51   Log-Likelihood:                -47790.
No. Observations:                4759   AIC:                         9.561e+04
Df Residuals:                    4745   BIC:                         9.570e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -5085.5684    323.427    -15.724      0.0

In [10]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import talib as ta
from talib import MA_Type

In [11]:
dt_all = pd.date_range(start=df.index[0],end=df.index[-1])

dt_obs = [d.strftime("%Y-%m-%d") for d in pd.to_datetime(df.index)]

dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d").tolist() if not d in dt_obs]

In [13]:
import plotly.graph_objects as go
fig = go.Figure(data=[go.Candlestick(x=df['Date'], open=df['Open'],
                                     high=df['High'], low=df['Low'],
                                     close=df['Close'])])

# Sets customized padding
fig.update_layout(margin=go.layout.Margin(r=10,b=10))

# Remove dates without values
fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

fig.update_yaxes(title_text='Price')
fig.update_xaxes(title_text='Date')

fig.update_layout(title = f"{stock}" + ' - CandleStick Chart',
                  xaxis_rangeslider_visible=False,
                  height=500, template='plotly_dark')

Chay RSI

In [15]:
# RSI
df['RSI'] = ta.RSI(df['Close'], 21)

# Construct a 1 x 1 Plotly figure
fig = make_subplots(rows=1, cols=1)

# Plot RSI
fig.add_trace(go.Scatter(x=df['Date'], y=df['RSI'], name='RSI'))

fig.add_hline(y=30, line_dash='dash', line_color='limegreen', line_width=1)
fig.add_hline(y=70, line_dash='dash', line_color='red', line_width=1)
fig.update_yaxes(title_text='RSI Score')

# Adds the range selector
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=[
                dict(count=1, label='1m', step='month', stepmode='backward'),
                dict(count=6, label='6m', step='month', stepmode='backward'),
                dict(count=1, label='YTD', step='year', stepmode='todate'),
                dict(count=1, label='1y', step='year', stepmode='backward'),
                dict(step='all')
            ]),
        type='date')
)

# Set the color from white to black on range selector buttons
fig.update_layout(xaxis=dict(rangeselector=dict(font=dict(color='black'))))

# Sets customized padding
fig.update_layout(margin=go.layout.Margin(r=10, b=10))

# Remove dates without values
fig.update_xaxes(rangebreaks=[dict(values=dt_breaks)])

layout = go.Layout(template='plotly_dark', title=f"{stock} - RSI", height=500, legend_title='Legend')
fig.update_layout(layout)

fig.show()

Boilinger Band

In [16]:
df['BU'], df['BM'], df['BL'] = ta.BBANDS(df.Close,
                                                                 timeperiod=20,
                                                                 matype=MA_Type.EMA)
fig = px.line(data_frame = df, x = df.index,
              y = ['Close', 'BU', 'BM', 'BL'])

# Update y & x axis labels
fig.update_yaxes(title_text='Price')
fig.update_xaxes(title_text='Date')
    
fig.data[0].name = 'Price'

# Sets customized padding
fig.update_layout(margin=go.layout.Margin(
        r=10,
        b=10))

layout = go.Layout(template='plotly_dark', 
                   title=f"{stock}" + ' - Price, Bollinger Bands',
                   height=500, legend_title='Legend')
fig.update_layout(layout)

In [17]:
# Drop Buy and Sell columns if they exist
df.drop(['Buy', 'Sell'], inplace=True, axis=1, errors='ignore')    

# Create DataFrame
df_buy = df.query('Low < BL')[['Date', 'Close']]
df_sell = df.query('High > BU')[['Date', 'Close']]

# Round close values for both buy and sell
df_buy['Close'] = round(df_buy.Close.round())
df_sell['Close'] = round(df_sell.Close.round())

In [19]:
fig = go.Figure(data=[go.Candlestick(x=df['Date'], open=df['Open'],
                                     high=df['High'],
                                     low=df['Low'],
                                      close=df['Close'],
                                      name='Candlestick')])

# Plot BU line graph; don't show legend
fig.add_trace(go.Scatter(x=df['Date'], y=df['BU'],
                         fill=None, mode='lines', showlegend=False))

# Plot BL line graph and fill upto BU; don't show legend
fig.add_trace(go.Scatter(x=df['Date'], y=df['BL'],
                         fill='tonexty', mode='lines', showlegend=False))

# Plot Buy signals
fig.add_trace(go.Scatter(x=df_buy['Date'], y=df_buy['Close'], mode='markers',
    marker=dict(symbol='x', size=7, line=dict(width=1)), name = 'Buy'))

# Plot Sell Signls
fig.add_trace(go.Scatter(x=df_sell['Date'], y=df_sell['Close'], mode='markers',
    marker=dict(symbol='diamond', size=7, line=dict(width=1)), name = 'Sell'))

fig.update_yaxes(title_text='Price')
fig.update_xaxes(title_text='Date')
    
fig.data[0].name = 'Price'
fig.update_layout(margin=go.layout.Margin(r=10,b=10))

layout = go.Layout(template='plotly_dark',
                   title =f"{stock}" + ' - Buy / Sell Signals', height=500,
                    xaxis_rangeslider_visible=False)
fig.update_layout(layout)

MACD

In [20]:
# Calculate MACD values
# Empty Data Frame to collect MACD analysis results
analysis = pd.DataFrame()
analysis['macd'], analysis['macdSignal'],analysis['macdHist'] = ta.MACD(df.Close,
                                                                fastperiod=12,
                                                                slowperiod=26,
                                                                signalperiod=9)
fig = make_subplots(rows=2, cols=1)

# Candlestick chart for pricing
fig.append_trace(go.Candlestick(x=df['Date'], open=df['Open'],
                                high=df['High'], low=df['Low'],
                                close=df['Close'], showlegend=False),
                                row=1, col=1)

# Fast Signal (%k)
fig.append_trace(go.Scatter(x=df['Date'],
                            y=analysis['macd'],
                            line=dict(color='#C42836', width=1),
                            name='MACD Line'), row=2, col=1)

# Slow signal (%d)
fig.append_trace(go.Scatter(x=df['Date'], y=analysis['macdSignal'],
                            line=dict(color='limegreen', width=1),
                            name='Signal Line'), row=2, col=1)

# Colorize the histogram values
colors = np.where(analysis['macd'] < 0, '#EA071C', '#57F219')

# Plot the histogram
fig.append_trace(go.Bar(x=df['Date'], y=analysis['macdHist'],
                        name='Histogram', marker_color=colors),
                       row=2, col=1)

fig['layout']['yaxis']['title']='Price'
fig['layout']['xaxis2']['title']='Date'

fig.data[0].name = 'Price'

# Sets customized padding
fig.update_layout(margin=go.layout.Margin(r=10, b=10)) 

# Make it pretty
layout = go.Layout(template='plotly_dark', title = f"{stock}" + ' - MACD Indicator', height=700,
    xaxis_rangeslider_visible=False)

# Update options and show plot
fig.update_layout(layout)

fig.update_layout(legend=dict(yanchor="top", y=0.45, xanchor="left", x=1.01))
fig.show()